In [1]:
from pyspark.sql import SparkSession

spark = SparkSession.builder \
.appName('Spark_Table_Cache') \
.getOrCreate()

25/07/23 10:47:25 WARN SparkSession: Using an existing Spark session; only runtime SQL configurations will take effect.


In [2]:
customers_schema = 'customers_id INT, name STRING, state STRING, registration_date STRING, is_active BOOLEAN'

In [3]:
customers_df = spark.read.format('csv').schema(customers_schema).load('/user/customers.csv')

In [10]:
customers_df.write.mode('ignore').saveAsTable('default.customers')

In [11]:
spark.sql('describe extended customers').show(truncate = False)

+----------------------------+-------------------------------------------------------------+-------+
|col_name                    |data_type                                                    |comment|
+----------------------------+-------------------------------------------------------------+-------+
|customers_id                |int                                                          |NULL   |
|name                        |string                                                       |NULL   |
|state                       |string                                                       |NULL   |
|registration_date           |string                                                       |NULL   |
|is_active                   |boolean                                                      |NULL   |
|                            |                                                             |       |
|# Detailed Table Information|                                                             

In [12]:
spark.sql('describe customers').show()

+-----------------+---------+-------+
|         col_name|data_type|comment|
+-----------------+---------+-------+
|     customers_id|      int|   NULL|
|             name|   string|   NULL|
|            state|   string|   NULL|
|registration_date|   string|   NULL|
|        is_active|  boolean|   NULL|
+-----------------+---------+-------+



In [13]:
!hadoop fs -ls -h /user/hive/warehouse/customers

Found 2 items
-rw-r--r--   2 root hadoop          0 2025-07-21 12:50 /user/hive/warehouse/customers/_SUCCESS
-rw-r--r--   2 root hadoop    172.4 K 2025-07-21 12:50 /user/hive/warehouse/customers/part-00000-d5b0e2cc-3028-478a-84a3-94302a016e23-c000.snappy.parquet


In [14]:
spark.sql('select * from customers limit 5').show()

+------------+----------+---------+-----------------+---------+
|customers_id|      name|    state|registration_date|is_active|
+------------+----------+---------+-----------------+---------+
|        NULL|      name|     city|            state|     NULL|
|           0|Customer_0|     Pune|      Maharashtra|     NULL|
|           1|Customer_1|Bangalore|       Tamil Nadu|     NULL|
|           2|Customer_2|Hyderabad|          Gujarat|     NULL|
|           3|Customer_3|Bangalore|        Karnataka|     NULL|
+------------+----------+---------+-----------------+---------+



In [15]:
spark.sql('select count(8) from customers').show()

+--------+
|count(8)|
+--------+
|   17654|
+--------+



In [16]:
spark.sql('cache table default.customers') # Eager Caching

25/07/23 10:50:48 WARN CacheManager: Asked to cache already cached data.


DataFrame[]

In [8]:
spark.sql('select count(*) from default.customers').show()

+--------+
|count(1)|
+--------+
|   17654|
+--------+



In [22]:
spark.sql('select state,count(*) from customers group by state').show()

+---------+--------+
|    state|count(1)|
+---------+--------+
|    Delhi|    2200|
|  Kolkata|    2223|
|Hyderabad|    2242|
|     city|       1|
|Bangalore|    2211|
|Ahmedabad|    2198|
|  Chennai|    2194|
|   Mumbai|    2142|
|     Pune|    2243|
+---------+--------+



In [23]:
spark.sql('uncache table customers')

DataFrame[]

In [24]:
spark.sql('cache lazy table customers')

DataFrame[]

In [25]:
spark.sql('select count(*) from customers').show()

+--------+
|count(1)|
+--------+
|   17654|
+--------+



In [26]:
spark.sql('show tables').show()

+---------+---------+-----------+
|namespace|tableName|isTemporary|
+---------+---------+-----------+
|  default|customers|      false|
+---------+---------+-----------+



In [27]:
spark.catalog.clearCache()

In [28]:
spark.stop()